In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error

In [2]:
m = pd.read_csv("../../data/processed/KOBIS_개봉일람_2.csv")

m

,영화명,감독,제작사,배급사,개봉일,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
0,명량,김한민,(주)빅스톤픽쳐스,(주)씨제이이엔엠,2014-07-30,한국,1587,1.357484e+11,17613682,3.312123e+10,4163666,사극,15세이상관람가,일반영화
1,극한직업,이병헌,(주)어바웃잇영화사 해그림 주식회사(주)씨제이이엔엠,(주)씨제이이엔엠,2019-01-23,한국,1978,1.396480e+11,16264944,3.185866e+10,3638287,코미디,15세이상관람가,일반영화
2,신과함께-죄와 벌,김용화,리얼라이즈픽쳐스(주)(주)덱스터스튜디오,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,한국,1912,1.156987e+11,14410754,2.753083e+10,3346172,판타지,12세이상관람가,일반영화
3,국제시장,윤제균,(주)제이케이필름(주)씨제이이엔엠,(주)씨제이이엔엠,2014-12-17,한국,966,1.108280e+11,14245998,2.584252e+10,3233946,드라마,12세이상관람가,일반영화
4,어벤져스: 엔드게임,안소니 루소조 루소,마블 스튜디오,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,미국,2835,1.221827e+11,13934592,3.357714e+10,3597963,액션,12세이상관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,늑대와 빨간 재킷,뱅상 파로노,NaN,판씨네마(주),2021-02-03,프랑스,74,8.525200e+06,950,2.893740e+06,345,스릴러,청소년관람불가,독립/예술영화
3971,마라가 큰 결정을 해야 해,로버트 루케틱,NaN,(주)영화사오원,2021-01-07,미국,79,7.475940e+06,856,2.830800e+06,321,멜로/로맨스,15세이상관람가,일반영화
3972,락다운 213주,아담 메이슨,플래티넘 듄스,그린나래미디어(주)(주)인터파크,2022-08-31,미국,61,7.664799e+06,845,3.573900e+06,381,스릴러,15세이상관람가,일반영화
3973,래미의 드래곤월드 구출작전,웨이밍 황,NaN,와이드 릴리즈(주),2020-07-16,중국,65,6.067340e+06,779,6.345000e+05,74,애니메이션,전체관람가,일반영화


In [3]:
m.isna().sum()

영화명           0
감독            3
제작사        1643
배급사           0
개봉일           0
국적            0
전국 스크린수       0
전국 매출액        0
전국 관객수        0
서울 매출액        0
서울 관객수        0
장르            0
등급            0
영화구분          0
dtype: int64

In [4]:
# 감독, 제작사 결측치 제거
m = m.dropna(subset=["감독", "제작사"])

m

,영화명,감독,제작사,배급사,개봉일,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
0,명량,김한민,(주)빅스톤픽쳐스,(주)씨제이이엔엠,2014-07-30,한국,1587,1.357484e+11,17613682,3.312123e+10,4163666,사극,15세이상관람가,일반영화
1,극한직업,이병헌,(주)어바웃잇영화사 해그림 주식회사(주)씨제이이엔엠,(주)씨제이이엔엠,2019-01-23,한국,1978,1.396480e+11,16264944,3.185866e+10,3638287,코미디,15세이상관람가,일반영화
2,신과함께-죄와 벌,김용화,리얼라이즈픽쳐스(주)(주)덱스터스튜디오,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,한국,1912,1.156987e+11,14410754,2.753083e+10,3346172,판타지,12세이상관람가,일반영화
3,국제시장,윤제균,(주)제이케이필름(주)씨제이이엔엠,(주)씨제이이엔엠,2014-12-17,한국,966,1.108280e+11,14245998,2.584252e+10,3233946,드라마,12세이상관람가,일반영화
4,어벤져스: 엔드게임,안소니 루소조 루소,마블 스튜디오,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,미국,2835,1.221827e+11,13934592,3.357714e+10,3597963,액션,12세이상관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3956,코드 8,제프 챈,XYZ 필름스,(주)팝엔터테인먼트,2020-07-02,기타,82,1.124136e+07,1362,3.337960e+06,400,SF,15세이상관람가,일반영화
3959,레이징 파이어,진목승,텐센트 픽처스,(주)제이앤씨미디어그룹(주)씨네필운,2022-01-20,홍콩,60,1.216316e+07,1346,5.719560e+06,605,액션,15세이상관람가,독립/예술영화
3963,섀도우 클라우드,로젠느 리앙,버티컬 엔터테인먼트,(주)제이앤씨미디어그룹,2021-04-22,미국,71,1.151214e+07,1282,3.775040e+06,384,SF,15세이상관람가,일반영화
3965,그녀의 비밀정원,김인식,(주)인디컴미디어,(유)조이앤시네마(주)제이앤씨미디어그룹,2020-04-22,한국,79,9.966880e+06,1254,2.905860e+06,375,드라마,15세이상관람가,독립/예술영화


In [5]:
# 감독별 전국 관객수 로그 변환 평균값 계산
director_groupby = m.groupby("감독")["전국 관객수"].apply(lambda x: np.log1p(x).mean())
# 제작사별 전국 관객수 로그 변환 평균값 계산
producer_groupby = m.groupby("제작사")["전국 관객수"].apply(lambda x: np.log1p(x).mean())
# 배급사별 전국 관객수 로그 변환 평균값 계산
distributor_groupby = m.groupby("배급사")["전국 관객수"].apply(lambda x: np.log1p(x).mean())

# 각 값을 변환
m["감독"] = m["감독"].map(director_groupby)
m["제작사"] = m["제작사"].map(producer_groupby)
m["배급사"] = m["배급사"].map(distributor_groupby)

m

C:\Users\user\AppData\Local\Temp\ipykernel_6800\4283574835.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m["감독"] = m["감독"].map(director_groupby)
C:\Users\user\AppData\Local\Temp\ipykernel_6800\4283574835.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m["제작사"] = m["제작사"].map(producer_groupby)
C:\Users\user\AppData\Local\Temp\ipykernel_6800\4283574835.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,영화명,감독,제작사,배급사,개봉일,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
0,명량,14.306190,12.886638,13.607103,2014-07-30,한국,1587,1.357484e+11,17613682,3.312123e+10,4163666,사극,15세이상관람가,일반영화
1,극한직업,15.172542,16.604523,13.607103,2019-01-23,한국,1978,1.396480e+11,16264944,3.185866e+10,3638287,코미디,15세이상관람가,일반영화
2,신과함께-죄와 벌,15.714698,16.402393,12.960970,2017-12-20,한국,1912,1.156987e+11,14410754,2.753083e+10,3346172,판타지,12세이상관람가,일반영화
3,국제시장,15.908342,15.263750,13.607103,2014-12-17,한국,966,1.108280e+11,14245998,2.584252e+10,3233946,드라마,12세이상관람가,일반영화
4,어벤져스: 엔드게임,14.740589,15.240150,13.427532,2019-04-24,미국,2835,1.221827e+11,13934592,3.357714e+10,3597963,액션,12세이상관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3956,코드 8,7.217443,7.217443,9.570018,2020-07-02,기타,82,1.124136e+07,1362,3.337960e+06,400,SF,15세이상관람가,일반영화
3959,레이징 파이어,7.205635,7.205635,7.205635,2022-01-20,홍콩,60,1.216316e+07,1346,5.719560e+06,605,액션,15세이상관람가,독립/예술영화
3963,섀도우 클라우드,7.156956,8.649415,9.092294,2021-04-22,미국,71,1.151214e+07,1282,3.775040e+06,384,SF,15세이상관람가,일반영화
3965,그녀의 비밀정원,7.770353,7.134891,7.134891,2020-04-22,한국,79,9.966880e+06,1254,2.905860e+06,375,드라마,15세이상관람가,독립/예술영화


In [6]:
# 개봉일에서 개봉월만 추출
m["개봉월"] = m["개봉일"].apply(lambda x: x.split("-")[1])

# 영화명, 개봉일 컬럼 삭제
m = m.drop(["영화명", "개봉일", "전국 매출액", "서울 매출액", "서울 관객수"], axis=1)

m

C:\Users\user\AppData\Local\Temp\ipykernel_6800\1802109519.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m["개봉월"] = m["개봉일"].apply(lambda x: x.split("-")[1])


,감독,제작사,배급사,국적,전국 스크린수,전국 관객수,장르,등급,영화구분,개봉월
0,14.306190,12.886638,13.607103,한국,1587,17613682,사극,15세이상관람가,일반영화,07
1,15.172542,16.604523,13.607103,한국,1978,16264944,코미디,15세이상관람가,일반영화,01
2,15.714698,16.402393,12.960970,한국,1912,14410754,판타지,12세이상관람가,일반영화,12
3,15.908342,15.263750,13.607103,한국,966,14245998,드라마,12세이상관람가,일반영화,12
4,14.740589,15.240150,13.427532,미국,2835,13934592,액션,12세이상관람가,일반영화,04
...,...,...,...,...,...,...,...,...,...,...
3956,7.217443,7.217443,9.570018,기타,82,1362,SF,15세이상관람가,일반영화,07
3959,7.205635,7.205635,7.205635,홍콩,60,1346,액션,15세이상관람가,독립/예술영화,01
3963,7.156956,8.649415,9.092294,미국,71,1282,SF,15세이상관람가,일반영화,04
3965,7.770353,7.134891,7.134891,한국,79,1254,드라마,15세이상관람가,독립/예술영화,04


In [7]:
# 영화형태, 국적, 장르, 등급, 영화구분 열을 더미 변수로 변환
dummy_columns = ["국적", "장르", "등급", "영화구분"]
m_dummy = pd.get_dummies(m[dummy_columns], prefix=dummy_columns)

# 더미 변수를 새로운 데이터프레임에 추가
m_new = pd.concat([m, m_dummy], axis=1)

# 기존의 영화형태, 국적, 장르, 등급, 영화구분 열 삭제
m_new.drop(dummy_columns, axis=1, inplace=True)

# 결과 확인
m_new

,감독,제작사,배급사,전국 스크린수,전국 관객수,개봉월,국적_기타,국적_독일,국적_러시아,국적_미국,...,장르_어드벤처,장르_전쟁,장르_코미디,장르_판타지,등급_12세이상관람가,등급_15세이상관람가,등급_전체관람가,등급_청소년관람불가,영화구분_독립/예술영화,영화구분_일반영화
0,14.306190,12.886638,13.607103,1587,17613682,07,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
1,15.172542,16.604523,13.607103,1978,16264944,01,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
2,15.714698,16.402393,12.960970,1912,14410754,12,False,False,False,False,...,False,False,False,True,True,False,False,False,False,True
3,15.908342,15.263750,13.607103,966,14245998,12,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
4,14.740589,15.240150,13.427532,2835,13934592,04,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3956,7.217443,7.217443,9.570018,82,1362,07,True,False,False,False,...,False,False,False,False,False,True,False,False,False,True
3959,7.205635,7.205635,7.205635,60,1346,01,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
3963,7.156956,8.649415,9.092294,71,1282,04,False,False,False,True,...,False,False,False,False,False,True,False,False,False,True
3965,7.770353,7.134891,7.134891,79,1254,04,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False


In [8]:
# 상관관계 확인
m_new.iloc[:, :6].corr()

,감독,제작사,배급사,전국 스크린수,전국 관객수,개봉월
감독,1.000000,0.864178,0.744978,0.646083,0.559955,0.005630
제작사,0.864178,1.000000,0.752690,0.636334,0.568284,-0.010484
배급사,0.744978,0.752690,1.000000,0.477674,0.411173,0.002857
전국 스크린수,0.646083,0.636334,0.477674,1.000000,0.651726,0.027905
전국 관객수,0.559955,0.568284,0.411173,0.651726,1.000000,0.036604
개봉월,0.005630,-0.010484,0.002857,0.027905,0.036604,1.000000


In [9]:
# 개봉월 컬럼 삭제
m_new.drop(["개봉월"], axis=1, inplace=True)

m_new

,감독,제작사,배급사,전국 스크린수,전국 관객수,국적_기타,국적_독일,국적_러시아,국적_미국,국적_스페인,...,장르_어드벤처,장르_전쟁,장르_코미디,장르_판타지,등급_12세이상관람가,등급_15세이상관람가,등급_전체관람가,등급_청소년관람불가,영화구분_독립/예술영화,영화구분_일반영화
0,14.306190,12.886638,13.607103,1587,17613682,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
1,15.172542,16.604523,13.607103,1978,16264944,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
2,15.714698,16.402393,12.960970,1912,14410754,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,True
3,15.908342,15.263750,13.607103,966,14245998,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
4,14.740589,15.240150,13.427532,2835,13934592,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3956,7.217443,7.217443,9.570018,82,1362,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
3959,7.205635,7.205635,7.205635,60,1346,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
3963,7.156956,8.649415,9.092294,71,1282,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
3965,7.770353,7.134891,7.134891,79,1254,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False


In [10]:
# 다중공선선 확인을 위해 더미 컬럼 제외
X = m_new.iloc[:, :5]

# 변수(features)와 VIF 값을 담을 데이터프레임 생성
vif = pd.DataFrame()
vif["features"] = X.columns

# X 데이터프레임의 각 변수에 대한 VIF 값을 계산하여 vif_factor 컬럼에 저장
vif["vif_factor"] = [variance_inflation_factor(X.values.astype(float), i) for i in range(X.shape[1])]
vif

,features,vif_factor
0,감독,178.515333
1,제작사,184.651319
2,배급사,122.706715
3,전국 스크린수,4.393730
4,전국 관객수,2.169640


In [11]:
# 상관관계 행렬
m_corr = m_new.iloc[:, :5].corr()

# 상관관계 행렬로부터 공분산행렬 구하기
cov_matrix = np.linalg.inv(m_corr.values)

# 공분산행렬에서 대각성분 계산
eigenvalues = np.linalg.eigvals(cov_matrix)

# 다중공선성 진단을 위한 조건수(condition number) 계산
cond_num = np.max(eigenvalues) / np.min(eigenvalues)

cond_num

26.206884591040787

In [12]:
# 변수 제거 함수
def recursive_feature_elimination(X, y, model, min_features=1, verbose=True):
    """
    재귀적 변수 제거를 수행하는 함수
    
    X: 독립 변수 데이터프레임
    y: 종속 변수 데이터프레임
    model: 모델 객체
    min_features: 최소 변수 개수
    verbose: 출력 여부
    """
    # 초기 변수 개수
    n_features = X.shape[1]
    
    # 현재 변수들의 성능
    best_score = float('inf')
    best_features = X.columns.tolist()
    
    # 최소 변수 개수 이상인 경우
    while n_features > min_features:
        # 모든 변수에 대해 반복
        scores = []
        for feature in X.columns:
            # 선택한 변수 제외
            features = X.columns.drop(feature)
            X_new = X[features]
            
            # 모델 학습 및 평가
            model.fit(X_new, y)
            y_pred = model.predict(X_new)
            score = mean_squared_error(y, y_pred)
            scores.append(score)
            
        # 가장 성능이 좋은 변수 선택
        idx = pd.Index(scores).argmin()
        worst_feature = X.columns[idx]
        
        # 변수 제거
        X.drop(worst_feature, axis=1, inplace=True)
        n_features = X.shape[1]
        if verbose:
            print(f'Removing {worst_feature}: {n_features} features left')
        
        # 현재 변수 개수에서의 성능이 최선인 경우
        if min(scores) < best_score:
            best_score = min(scores)
            best_features = X.columns.tolist()
        else:
            break
    
    # 최종 선택된 변수들
    return best_features

# ElasticNetCV 모델 객체 생성
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
enet = ElasticNetCV(alphas=alphas, l1_ratio=0.5, cv=5)

# 최소 변수 개수 지정
min_features = 5

# 재귀적 변수 제거 수행
X = m_new.drop('전국 관객수', axis=1)
y = m_new['전국 관객수']
selected_features = recursive_feature_elimination(X, y, enet, min_features=min_features)

# 선택된 변수 출력
print(f'{len(selected_features)} features selected:')
print(selected_features)


c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 29518004093753.84, tolerance: 16523375746.153503
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1733018701439647.2, tolerance: 804146052204.6497
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1930413830633997.5, tolerance: 779389215051.8561
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packa

Removing 국적_기타: 40 features left


c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26763999935933.312, tolerance: 16523375746.153503
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1717173689219251.5, tolerance: 804146052204.6497
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1915457250362077.0, tolerance: 779389215051.8561
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-pack

Removing 국적_프랑스: 39 features left


c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18658547004456.15, tolerance: 16523375746.153503
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1754248083271489.5, tolerance: 804146052204.6497
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1951851265097466.0, tolerance: 779389215051.8561
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packa

Removing 국적_홍콩: 38 features left
39 features selected:
['감독', '제작사', '배급사', '전국 스크린수', '국적_독일', '국적_러시아', '국적_미국', '국적_스페인', '국적_영국', '국적_일본', '국적_중국', '국적_한국', '국적_홍콩', '장르_SF', '장르_가족', '장르_공연', '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마', '장르_멜로/로맨스', '장르_뮤지컬', '장르_미스터리', '장르_범죄', '장르_사극', '장르_서부극(웨스턴)', '장르_스릴러', '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁', '장르_코미디', '장르_판타지', '등급_12세이상관람가', '등급_15세이상관람가', '등급_전체관람가', '등급_청소년관람불가', '영화구분_독립/예술영화', '영화구분_일반영화']


In [13]:
m_new

,감독,제작사,배급사,전국 스크린수,전국 관객수,국적_기타,국적_독일,국적_러시아,국적_미국,국적_스페인,...,장르_어드벤처,장르_전쟁,장르_코미디,장르_판타지,등급_12세이상관람가,등급_15세이상관람가,등급_전체관람가,등급_청소년관람불가,영화구분_독립/예술영화,영화구분_일반영화
0,14.306190,12.886638,13.607103,1587,17613682,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
1,15.172542,16.604523,13.607103,1978,16264944,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
2,15.714698,16.402393,12.960970,1912,14410754,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,True
3,15.908342,15.263750,13.607103,966,14245998,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
4,14.740589,15.240150,13.427532,2835,13934592,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3956,7.217443,7.217443,9.570018,82,1362,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
3959,7.205635,7.205635,7.205635,60,1346,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
3963,7.156956,8.649415,9.092294,71,1282,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
3965,7.770353,7.134891,7.134891,79,1254,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False


In [14]:
m_new.to_csv("../../data/processed/KOBIS_개봉일람_3.csv", index=False)